# Introduction

Aujourd'hui la plupart des sites web utilisent du JavaScript pour rendre le site plus dynamique et agréable mais aussi pour afficher la données. 

Pour ces sites, la première méthode abordée n'est pas efficace, car de nombreuses requêtes sont utilisées pour se connecter, afficher la donnée, naviguer, il faudrait prendre en compte toutes ces requêtes et les assembler pour simuler le comportement du site. 

**C'est le comportement de votre navigateur**, il compile toutes les interactions, les requêtes et génère du HTML permettant d'afficher l'interface graphique d'un site web.

Pour palier ce problème on peut laisser le navigateur faire son travail et contrôler sa surcouche logiciel. Pour cela on peut prendre le contrôle d'un navigateur comme Chrome, Edge, Firefox ou Safari depuis un script Python. 

Pour cela, nous allons utiliser un package Python `Selenium` https://selenium-python.readthedocs.io/ qui permet d'instancier un navigateur et de le contrôler. Sélénium est beaucoup utilisé pour générer des tests automatiques de sites web. 

On peut trouver les drivers des différents browser : 
- Chrome:	https://sites.google.com/a/chromium.org/chromedriver/downloads
- Edge:	https://developer.microsoft.com/en-us/microsoft-edge/tools/webdriver/
- Firefox:	https://github.com/mozilla/geckodriver/releases
- Safari:	https://webkit.org/blog/6900/webdriver-support-in-safari-10/

In [1]:
!pip install selenium


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\keren\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
from selenium import webdriver

In [3]:
chrome = webdriver.Chrome()

Vous devez voir apparaître une nouvelle instance de votre navigateur. Si vous voulez accéder à une page : 

In [4]:
chrome.get("https://www.amazon.fr")

Allez maintenant voir votre fenêtre. Vous êtes sur le site d'amazon. Vous pouvez maintenant vous déplacer dans la page. Pour cela il va falloir inspecter le code source de la page.

Il existe de nombreuse méthode pour récupérer les éléments par exemple `find_element_by_link_text` qui permet de récupérer un élément grâce au text affiché. Ici, si on veut récupérer toutes les `Ventes Flash`

In [6]:
from selenium.webdriver.common.by import By
from selenium.common.exceptions import WebDriverException

# Clique sur le bouton "Continuer sans accepter"
try:
    link = chrome.find_element(By.LINK_TEXT, "Continuer sans accepter")
    link.click()
except WebDriverException:
    pass  

# Utilise XPATH pour trouver "Ventes Flash"
try:
    link = chrome.find_element(By.XPATH, "//a[contains(text(), 'Ventes Flash')]")
    type(link)
except WebDriverException as e:
    print(f"Error: {e}")

# Recherche par texte (By.LINK_TEXT) échoue car s'attend à trouver exactement "Ventes Flash"
# Ainsi XPath contains(text(), 'Ventes Flash') permet lui de trouver


Si on veut entrer dans le lien, il suffit d'appeler la méthode `click`.

In [7]:
link.click()

Vous êtes arrivé sur la page des ventes flash. Maintenant nous allons récupérer toutes les ventes actuelles. On remarque que les ventes sont stockés dans des widgets qui porte l'id `widgetContent`.

In [8]:
# widgetContent plus maintenant..

#all_widgets = chrome.find_elements(By.ID, "widgetContent") 
#type(all_widgets), len(all_widgets)

On obtient une liste d'éléments décrits par cet identifiant. Il y en a 5, un pour chaque ligne. Pour récupérer les différentes offres de la première ligne. on récupère les sections correspondantes: 

In [78]:
#first_line = all_widgets[0]
#all_first_line_elements = first_line.find_elements_by_class_name('a-section')
#len(all_first_line_elements)

Pour récupérer le text maintenant : 

In [82]:
#_ = [print(elt.text+ "\n")  for elt in all_first_line_elements]

In [9]:
carousel_element = chrome.find_element(By.CSS_SELECTOR, '.a-carousel')

# 
# Trouver les éléments enfants de l'élément via ici 'a'
all_links_in_carousel = carousel_element.find_elements(By.TAG_NAME, 'a')

print("Voici les Ventes Flash :\n")
for link in all_links_in_carousel:
    text=(link.text).replace("\n", "").replace("Vente flash","").replace("Le plus bas : ","")
    if "%" in text:
        text=text.split('%')
        reduc=text[0]+'%'
        
        if len(text)>1:
            text=text[1].split('€')
            if text :
                t=text[0]
                l=len(t)
                prices=t[:l-2]+","+t[l-2:]+"€"
                price=text[1]+"€"

                if len(text)>2:
                    titre=text[2]
                    lien=link.get_attribute("href")
                    print(titre," :\n","Prix en solde :",prices,",avec",reduc," de réduction\n", "=> Prix de base :",price,"\nLien :",lien)
    
    print("\n")

Voici les Ventes Flash :

Philips Epilateur Satinelle Essentiel comportant 21 em…  :
 Prix en solde : 17,99€ ,avec -31%  de réduction
 => Prix de base : 26,00€ 
Lien : https://www.amazon.fr/Philips-BRE225-00-Epilateur-Satinelle/dp/B07M86Y5J2/?_encoding=UTF8&_encoding=UTF8&ref_=dlx_deals_sc_dcl_img_dt_dealz_tdwdg&pd_rd_w=pafbM&content-id=amzn1.sym.34382300-6001-4869-b7f8-c73eebde5d45&pf_rd_p=34382300-6001-4869-b7f8-c73eebde5d45&pf_rd_r=12VW52DMHH4G9E04HPCN&pd_rd_wg=LM2lx&pd_rd_r=83f6911a-d58f-4a56-9116-4853128e106c


Philips Série 3000 Tondeuse cheveux et Multi-Styles com…  :
 Prix en solde : 29,99€ ,avec -21%  de réduction
 => Prix de base : 37,99€ 
Lien : https://www.amazon.fr/MG3740-15-Multigroom-Multi-Styles-Accessoires/dp/B074MGQF4K/?_encoding=UTF8&_encoding=UTF8&ref_=dlx_deals_sc_dcl_img_dt_dealz_tdwdg&pd_rd_w=pafbM&content-id=amzn1.sym.34382300-6001-4869-b7f8-c73eebde5d45&pf_rd_p=34382300-6001-4869-b7f8-c73eebde5d45&pf_rd_r=12VW52DMHH4G9E04HPCN&pd_rd_wg=LM2lx&pd_rd_r=83f6911a-d58

Si vous êtes fan de JavaScript vous pouvez même injecter du code JS dans le navigateur depuis Python pour exécuter des opérations complexes. 

In [10]:
js_script = """
const class_name = 'DRIOC'
console.log(`Hi from ${class_name}`)
"""

In [11]:
chrome.execute_script(js_script)

In [27]:
chrome.close()

WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=119.0.6045.200)
Stacktrace:
	GetHandleVerifier [0x00007FF7A79482B2+55298]
	(No symbol) [0x00007FF7A78B5E02]
	(No symbol) [0x00007FF7A77705AB]
	(No symbol) [0x00007FF7A775D1AA]
	(No symbol) [0x00007FF7A775D9CE]
	(No symbol) [0x00007FF7A7770AF8]
	(No symbol) [0x00007FF7A774FB90]
	(No symbol) [0x00007FF7A77DC714]
	(No symbol) [0x00007FF7A77D2070]
	(No symbol) [0x00007FF7A77A670A]
	(No symbol) [0x00007FF7A77A7964]
	GetHandleVerifier [0x00007FF7A7CC0AAB+3694587]
	GetHandleVerifier [0x00007FF7A7D1728E+4048862]
	GetHandleVerifier [0x00007FF7A7D0F173+4015811]
	GetHandleVerifier [0x00007FF7A79E47D6+695590]
	(No symbol) [0x00007FF7A78C0CE8]
	(No symbol) [0x00007FF7A78BCF34]
	(No symbol) [0x00007FF7A78BD062]
	(No symbol) [0x00007FF7A78AD3A3]
	BaseThreadInitThunk [0x00007FFC3203257D+29]
	RtlUserThreadStart [0x00007FFC327AAA58

Vous pouvez aller vérifier dans la console Javascript du navigateur que le message apparait bien. 

# Exercice

## Exercice 1

Extraire le prix des offres de cette page : https://www.amazon.fr/s?k=pc+portable+gamer&crid=1RDZ1KDS76GLF&sprefix=pc+por%2Caps%2C72&ref=nb_sb_ss_ts-doa-p_1_6

a-section a-spacing-base div class
s-image img class
a-section a-spacing-small puis-padding-left-small puis-padding-right-small div class


In [20]:
#def extract_price(offer_web_element):
#    raise NotImplementedError('Need to be implemented')
#    pass

#extract_price(all_first_line_elements[0])

Extraire l'image de l'offre: 

In [14]:
#def extract_image(offer_web_element):
#    raise NotImplementedError('Need to be implemented')
#    pass

#extract_image(all_first_line_elements[0])

Extraire le titre de l'offre: 

In [13]:
#def extract_title(offer_web_element):
#    raise NotImplementedError('Need to be implemented')
#    pass

#extract_title(all_first_line_elements[0])

In [26]:
# Initialisation du navigateur (dans cet exemple, nous utilisons Chrome)
driver = webdriver.Chrome()

# Ouvrir la page Amazon
url = "https://www.amazon.fr/s?k=pc+portable+gamer&crid=1RDZ1KDS76GLF&sprefix=pc+por%2Caps%2C72&ref=nb_sb_ss_ts-doa-p_1_6"
driver.get(url)

try:
    # Récupérer toutes les sections contenant les informations sur les offres
    offre_sections = driver.find_elements(By.CSS_SELECTOR, 'div.s-result-item')

    # Parcourir chaque section d'offre
    for offre_section in offre_sections:
        # Récupérer l'image de l'offre
        image_offre = offre_section.find_element(By.CSS_SELECTOR, 'div.s-image img').get_attribute('src')

        # Récupérer les informations sur l'offre (titre, prix, note, etc)
        infos_offre = offre_section.find_element(By.CSS_SELECTOR, 'div.a-section.a-spacing-small div').text

        # Afficher les informations
        print("Image:", image_offre)
        print("Informations:", infos_offre)
        print("\n" + "="*50 + "\n")

except WebDriverException as e:
    print("Une exception s'est produite:", e)



In [ ]:
driver.close()